In [84]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

import keras as K
from keras.models import Sequential
from keras.layers import Embedding, GRU, LSTM, Dense, Input, TimeDistributed, Activation
from keras.optimizers import RMSprop
np.set_printoptions(precision=4, linewidth=200)

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from utils.reader import ptb_raw_data
from utils.batcher import generate_epoch

In [91]:
EMBEDDING_SIZE=128
HIDDEN_SIZE=128
BATCH_SIZE=16
NUM_STEPS=16
NUM_EPOCHS_INIT_LR=4
NUM_EPOCHS_TOTAL=12
INITIAL_LR=3e-4
LR_DECAY_RATE=0.75
MAX_NORM=5

In [92]:
def keras_generate_epoch(data, bs, ns):
    for x, y in generate_epoch(data, bs, ns):
        yield x, np.expand_dims(y, -1)
        

In [93]:
X_train, X_val, X_test, vocab_size = ptb_raw_data('bigdata/simple-examples/data')

In [94]:
optimizer = RMSprop(lr=1e-2)

model = Sequential()
model.add(Embedding(vocab_size, output_dim=EMBEDDING_SIZE, input_length=NUM_STEPS))
model.add(LSTM(HIDDEN_SIZE, return_sequences=True))
model.add(GRU(HIDDEN_SIZE, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (None, 16, 128)           1280000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 16, 128)           131584    
_________________________________________________________________
gru_30 (GRU)                 (None, 16, 128)           98688     
_________________________________________________________________
time_distributed_9 (TimeDist (None, 16, 10000)         1290000   
Total params: 2,800,272
Trainable params: 2,800,272
Non-trainable params: 0
_________________________________________________________________


In [95]:
for i in range(5):
    generator = keras_generate_epoch(X_train, BATCH_SIZE, NUM_STEPS)
    model.fit_generator(
        generator,
        steps_per_epoch=((len(X_train) // BATCH_SIZE - 1)// NUM_STEPS),
    )
    val_gen = keras_generate_epoch(X_val, BATCH_SIZE, NUM_STEPS)
    val_loss = model.evaluate_generator(
        val_gen,
        steps=((len(X_val) // BATCH_SIZE - 1)// NUM_STEPS)
    )
    print('val perplexity: {0:.1f}'.format(np.exp(val_loss)))

Epoch 1/1
3631/3631 [==============================] - 156s 43ms/step - loss: 6.1249
val perplexity: 546.7
Epoch 1/1
1263/3631 [=========>....................] - ETA: 1:41 - loss: 6.2265

KeyboardInterrupt: 

In [45]:
np.exp(9.21)

9996.5968594378755

In [70]:
x.shape

(128, 8)

In [71]:
y.shape

(128, 8)

In [72]:
y_pred = model.predict(x)

ValueError: Error when checking : expected embedding_19_input to have shape (None, 16) but got array with shape (128, 8)

In [73]:
my_gen = keras_generate_epoch(X_train, BATCH_SIZE, NUM_STEPS)

In [74]:
x, y = next(my_gen)

In [75]:
y_pred = model.predict(x)

In [76]:
y_pred.shape

(32, 16, 10000)

In [77]:
y_pred[0,0]

array([  1.0005e-04,   9.9916e-05,   1.0011e-04, ...,   9.9993e-05,   1.0000e-04,   1.0004e-04], dtype=float32)

In [78]:
np.sum(y_pred[0,0])

1.0